# Aerospace Spare Parts Optimization

This notebook formulates and solves a **multi-period aerospace spare parts distribution problem** using **Mixed Integer Linear Programming (MILP)**.

We use the `PuLP` Python library to:
- Minimize the total cost (transport, purchase, holding, shortage, and fixed ordering).
- Manage spare parts inventories at bases.
- Satisfy demand at multiple stations.
- Handle truck capacity limits and optional air shipments.
- Incorporate repair returns and order lead times.

This model reflects **real-world aerospace logistics challenges** in maintaining operational readiness.

In [1]:
# transport_spare_parts_milp.ipynb
import pulp
import time
import io
import sys
from tabulate import tabulate

ModuleNotFoundError: No module named 'tabulate'

## 1. Problem Data

We define:
- Bases: where parts are stored and shipped from.
- Stations: demand points for spare parts.
- Parts: two spare parts (`P1`, `P2`).
- Periods: 3 planning periods.

The dataset includes:
- Initial inventories.
- Demand forecasts.
- Transport costs (truck and air).
- Truck capacity.
- Purchase, holding, and shortage costs.
- Repair return fractions.

In [ ]:
Bases = ["Delhi", "Mumbai", "Bengaluru"]
Stations = ["Kolkata", "Hyderabad", "Chennai"]
Parts = ["P1", "P2"]
Periods = [1, 2, 3]

# Initial inventories
init_inv = {
    ("Delhi", "P1"): 10, ("Delhi", "P2"): 5,
    ("Mumbai", "P1"): 8, ("Mumbai", "P2"): 8,
    ("Bengaluru", "P1"): 5, ("Bengaluru", "P2"): 10,
}

# Demand forecasts
demand = {
    1: { "Kolkata": {"P1": 8,  "P2": 6},
         "Hyderabad": {"P1": 12, "P2": 10},
         "Chennai": {"P1": 10, "P2": 12}},
    2: { "Kolkata": {"P1": 10, "P2": 8},
         "Hyderabad": {"P1": 11, "P2": 11},
         "Chennai": {"P1": 12, "P2": 9}},
    3: { "Kolkata": {"P1": 9,  "P2": 10},
         "Hyderabad": {"P1": 13, "P2": 12},
         "Chennai": {"P1": 11, "P2": 13}},
}

# Transport costs
truck_cost = {
    ("Delhi","Kolkata"): 1.8, ("Delhi","Hyderabad"): 2.0, ("Delhi","Chennai"): 2.2,
    ("Mumbai","Kolkata"): 2.1, ("Mumbai","Hyderabad"): 1.9, ("Mumbai","Chennai"): 2.0,
    ("Bengaluru","Kolkata"): 2.6, ("Bengaluru","Hyderabad"): 1.8, ("Bengaluru","Chennai"): 1.6,
}
air_cost = {k: v + 1.5 for k,v in truck_cost.items()}

# Truck capacities
truck_capacity = {"Delhi": 25, "Mumbai": 20, "Bengaluru": 18}

# Cost parameters
purchase_cost = {"P1": 5.0, "P2": 7.0}
fixed_order_cost = 50.0
holding_cost = {"P1": 0.2, "P2": 0.25}
shortage_cost = {"P1": 20.0, "P2": 25.0}

# Repair return fractions
repair_return = {"P1": 0.20, "P2": 0.10}

# Big-M constant
BIGM = 1e6

## 2. Model Formulation

- **Decision variables:**
  - Orders placed from suppliers.
  - Binary indicator for fixed order cost.
  - Shipments via truck and air.
  - Inventory at bases.
  - Shortages and consumed quantities at stations.

- **Objective:** Minimize total costs.
- **Constraints:**
  1. Order binary linking.
  2. Truck capacity.
  3. Inventory balance (with repair returns).
  4. Demand satisfaction (consumed + shortage).
  5. Flow conservation.

In [ ]:
# Define model
model = pulp.LpProblem("Aerospace_Spare_Parts_MILP", pulp.LpMinimize)

# Decision variables
order_qty = pulp.LpVariable.dicts("OrderQty", ((b,p,t) for b in Bases for p in Parts for t in Periods), lowBound=0)
y_order   = pulp.LpVariable.dicts("Y_Order",  ((b,t) for b in Bases for t in Periods), lowBound=0, upBound=1, cat="Binary")
ship_truck= pulp.LpVariable.dicts("ShipTruck",((b,s,p,t) for b in Bases for s in Stations for p in Parts for t in Periods), lowBound=0)
ship_air  = pulp.LpVariable.dicts("ShipAir",  ((b,s,p,t) for b in Bases for s in Stations for p in Parts for t in Periods), lowBound=0)
inventory = pulp.LpVariable.dicts("Inv",     ((b,p,t) for b in Bases for p in Parts for t in Periods), lowBound=0)
shortage  = pulp.LpVariable.dicts("Shortage",((s,p,t) for s in Stations for p in Parts for t in Periods), lowBound=0)
consumed  = pulp.LpVariable.dicts("Consumed",((s,p,t) for s in Stations for p in Parts for t in Periods), lowBound=0)

In [ ]:
# Objective function
transport_cost = pulp.lpSum([truck_cost[(b,s)] * ship_truck[(b,s,p,t)] + air_cost[(b,s)] * ship_air[(b,s,p,t)]
                             for b in Bases for s in Stations for p in Parts for t in Periods])
purchase_cost_total = pulp.lpSum([purchase_cost[p] * order_qty[(b,p,t)] for b in Bases for p in Parts for t in Periods])
fixed_order_total   = pulp.lpSum([fixed_order_cost * y_order[(b,t)] for b in Bases for t in Periods])
holding_cost_total  = pulp.lpSum([holding_cost[p] * inventory[(b,p,t)] for b in Bases for p in Parts for t in Periods])
shortage_cost_total = pulp.lpSum([shortage_cost[p] * shortage[(s,p,t)] for s in Stations for p in Parts for t in Periods])

model += (transport_cost + purchase_cost_total + fixed_order_total + holding_cost_total + shortage_cost_total)

### Constraints

In [ ]:
# 1) Order linking
for b in Bases:
    for t in Periods:
        model += pulp.lpSum([order_qty[(b,p,t)] for p in Parts]) <= BIGM * y_order[(b,t)]

# 2) Truck capacity
for b in Bases:
    for t in Periods:
        model += pulp.lpSum([ship_truck[(b,s,p,t)] for s in Stations for p in Parts]) <= truck_capacity[b]

# 3) Inventory balance with returns
for b in Bases:
    for p in Parts:
        total_init = sum(init_inv.get((bb,p), 0) for bb in Bases)
        base_share = init_inv.get((b,p), 0) / total_init if total_init > 0 else 1/len(Bases)
        for t in Periods:
            arrivals = order_qty[(b,p,t-1)] if t-1 >= 1 else 0
            returns = repair_return[p] * base_share * pulp.lpSum([consumed[(s,p,t-1)] for s in Stations]) if t-1 >= 1 else 0
            outflows = pulp.lpSum([ship_truck[(b,s,p,t)] + ship_air[(b,s,p,t)] for s in Stations])
            if t == 1:
                model += init_inv.get((b,p), 0) + arrivals + returns - outflows == inventory[(b,p,t)]
            else:
                model += inventory[(b,p,t-1)] + arrivals + returns - outflows == inventory[(b,p,t)]

# 4) Demand satisfaction
for t in Periods:
    for s in Stations:
        for p in Parts:
            model += consumed[(s,p,t)] + shortage[(s,p,t)] == demand[t][s][p]

# 5) Flow conservation
for t in Periods:
    for s in Stations:
        for p in Parts:
            model += pulp.lpSum([ship_truck[(b,s,p,t)] + ship_air[(b,s,p,t)] for b in Bases]) == consumed[(s,p,t)]

## 3. Solve the Model

In [ ]:
solver = pulp.PULP_CBC_CMD(msg=1, timeLimit=60)
start = time.time()
old_stdout = sys.stdout
sys.stdout = mystdout = io.StringIO()
result = model.solve(solver)
sys.stdout = old_stdout
end = time.time()
cbc_log = mystdout.getvalue().splitlines()

print("Status:", pulp.LpStatus[model.status])
print(f"Objective (Total cost, ₹k): {pulp.value(model.objective):.2f}")
print(f"Solve Time: {end - start:.3f} seconds")

## 4. Results Summary

In [ ]:
# Orders
order_table = []
for b in Bases:
    for t in Periods:
        for p in Parts:
            q = order_qty[(b,p,t)].value()
            if q and q > 1e-6:
                order_table.append([b,t,p,q])
print("Orders placed:")
print(tabulate(order_table, headers=["Base","Period","Part","OrderQty"], floatfmt=".1f")) if order_table else print("No orders placed.")

# Shipments
ship_rows = []
for b in Bases:
    for s in Stations:
        for p in Parts:
            for t in Periods:
                st = ship_truck[(b,s,p,t)].value() or 0.0
                sa = ship_air[(b,s,p,t)].value() or 0.0
                if st > 1e-6 or sa > 1e-6:
                    ship_rows.append([b,s,p,t,st,sa])
print("\nShipments:")
print(tabulate(ship_rows, headers=["Base","Station","Part","Period","Truck","Air"], floatfmt=".1f"))

# Inventories
inv_rows = []
for b in Bases:
    for p in Parts:
        for t in Periods:
            inv = inventory[(b,p,t)].value() or 0.0
            if inv > 1e-6:
                inv_rows.append([b,p,t,inv])
print("\nInventory levels:")
print(tabulate(inv_rows, headers=["Base","Part","Period","Inv_End"], floatfmt=".1f"))

# Shortages
sh_rows = []
for s in Stations:
    for p in Parts:
        for t in Periods:
            sh = shortage[(s,p,t)].value() or 0.0
            if sh > 1e-6:
                sh_rows.append([s,p,t,sh])
print("\nShortages:")
print(tabulate(sh_rows, headers=["Station","Part","Period","Shortage"], floatfmt=".1f")) if sh_rows else print("No shortages.")